<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/5%20Procesado%20del%20lenguaje%20natural/Sesion%202/NLP_06d_Extraccio%CC%81n_caracteri%CC%81sticas_densas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word embeddings
## WE en spaCy
Los word embeddings (o word vectors) son representaciones numéricas de las palabras, generadas con una reducción de dimensionalidad sobre una matriz de co-ocurrencia sobre un corpus enorme. Spacy utiliza los word vectors de GloVe, (*Stanford's Global Vectors for Word Representation*). Estos vectores se pueden utilizar para calcular la similaridad semántica entre palabras o documentos.

El vocabulario por defecto en el modelo spaCy del idioma inglés (`en_core_web_sm`) es muy pequeño. Hay que cargar en_core_web_md (`python -m spacy download en_core_web_md`) para tener un conjunto de word vectors mayor. El modelo de tamaño medio en español (`python -m spacy download es_core_news_md`) contiene vectores también.

In [ ]:
import spacy
nlp = spacy.load("es_core_news_md")

In [ ]:
nlp.vocab.vectors

In [ ]:
len(nlp.vocab.vectors)

In [ ]:
nlp.vocab.vectors_length

In [ ]:
madrid = nlp.vocab["Madrid"]
madrid.vector.shape

In [ ]:
type(madrid)

In [ ]:
doc = nlp("me voy a Madrid")

In [ ]:
doc[3]

In [ ]:
doc[3].vector #equivale a nlp.get_vector("Madrid")

In [ ]:
nlp.vocab.get_vector("Madrid")

In [ ]:
#los tokens tienen el mismo lexema
madrid.vector == doc[3].vector

### Similitud semántica
Podemos calcular la similitud entre palabras mediante la *similitud coseno*

In [ ]:
toledo = nlp.vocab["Toledo"]
madrid.similarity(toledo)

In [ ]:
manzana = nlp.vocab["manzana"]
madrid.similarity(manzana)

In [ ]:
pera = nlp.vocab["pera"]
pera.similarity(manzana)

In [ ]:
nlp_en = spacy.load('en_core_web_md')

In [ ]:
len(nlp_en.vocab.vectors)

In [ ]:
nlp_en.vocab.vectors

`spaCy` no precarga el vocabulario para los modelos con vectores (a partir de la v2.3)

In [ ]:
nlp_en.vocab.vectors_length

Los lexemas se cargan conforme se usan en el texto. Pero queremos cargar todos los lexemas podemos recorrer todo el vocabulario con:

In [ ]:
len(nlp_en.vocab)

In [ ]:
for orth in nlp_en.vocab.vectors:
    _ = nlp_en.vocab[orth]

In [ ]:
len(nlp_en.vocab)

Podemos buscar términos similares/relacionados por similitud de sus vectores en el espacio vectorial:

In [ ]:
nlp_en = spacy.load('en_core_web_md')

nasa = nlp_en.vocab['NASA']

# cogemos todas las palabras del vocabulario que tienen vector, en minúsculas
# a partir de spaCy 2.3 no se puede recorrer el vocabulario con nlp.vocab sino nlp.vocab.vectors
# https://spacy.io/usage/v2-3
allWords = list({nlp_en.vocab[l] for l in nlp_en.vocab.vectors
                 if nlp_en.vocab[l].has_vector
 #                and nlp_en.vocab[l].orth_.islower()
                 and nlp_en.vocab[l].lower_ != "nasa"})


print("longitud:",len(allWords))
    
# ordenamos por similitud con NASA
allWords.sort(key=lambda w: nasa.similarity(w))
allWords.reverse()
print("Top 20 palabras más similares a NASA:")
for word in allWords[:20]:   
    print(word.orth_)

### Analogías entre palabras
Podemos hacer operaciones aritméticas con los vectores para buscar palabras relacionadas:  

In [ ]:
import numpy as np

# definimos similitud mediante distancia coseno (la que usa spaCy)
cosine = lambda v1, v2: np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
# Buscamos resolver la analogía:
# Man is to King as Woman is to ??
king = nlp_en.vocab['king']
man = nlp_en.vocab['man']
woman = nlp_en.vocab['woman']

#calculamos vector resultado
result = king.vector - man.vector + woman.vector

# listamos todas las palabras
allWords = list({w for w in nlp_en.vocab if w.has_vector and
                 w.orth_.islower() and w.lower_ != "king" and
                 w.lower_ != "man" and w.lower_ != "woman"})
#sólo funciona después de haber inicializado 

# ordenamos por similitud con el vector resultado
allWords.sort(key=lambda w: cosine(w.vector, result))
allWords.reverse()
print("\n----------------------------\nTop 3 resultados más similares para: king - man + woman:")
for word in allWords[:3]:   
    print(word.orth_)

### Visualización de word embeddings
Vamos a visualizar una proyección 2D de todo el espacio vectorial de *word embeddings*

In [ ]:
len(nlp.vocab)

In [ ]:
lexemas = [nlp.vocab[orth] for orth in nlp.vocab.vectors]

In [ ]:
len(lexemas)

In [ ]:
words = [t.text for t in np.random.choice(lexemas, 25, replace=False)]
word_vectors = np.array([nlp(word).vector for word in words])

words

### Visualización t-SNE

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(n_components=2, random_state=0, n_iter=10000, perplexity=2)
np.set_printoptions(suppress=True)
T = tsne.fit_transform(word_vectors)
labels = words
plt.figure(figsize=(14, 8))
plt.scatter(T[:, 0], T[:, 1], c='steelblue', edgecolors='k')
for label, x, y in zip(labels, T[:, 0], T[:, 1]):
    plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')

### Visualización PCA

In [ ]:
word_vectors = [t.vector for t in np.random.choice(lexemas, 10000, replace=False)]

In [ ]:
palabras = ['manzana', 'pera', 'Madrid', 'Toledo']

In [ ]:
palabras_vectors = np.array([nlp(word).vector for word in palabras])

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
np.set_printoptions(suppress=True)
T = pca.fit_transform(word_vectors)

plt.figure(figsize=(14, 8))
plt.scatter(T[:, 0], T[:, 1], c='steelblue',alpha=0.05)

labels = palabras
T = pca.transform(palabras_vectors)
plt.scatter(T[:, 0], T[:, 1], c='lime', edgecolors='darkgreen')

for label, x, y in zip(labels, T[:, 0], T[:, 1]):
    plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')

### Visualización t-SNE extendida

In [ ]:
palabras_all = [t.text for t in np.random.choice(lexemas, 10000, replace=False)] + palabras

In [ ]:
palabras_vectors = np.array([nlp(word).vector for word in palabras_all])

In [ ]:
tsne = TSNE(n_components=2, random_state=0, n_iter=250, perplexity=2)
np.set_printoptions(suppress=True)
T = tsne.fit_transform(palabras_vectors)

plt.figure(figsize=(14, 8))
plt.scatter(T[:, 0], T[:, 1], c='steelblue', alpha=0.05)

labels = palabras

plt.scatter(T[-len(palabras):, 0], T[-len(palabras):, 1], c='lime', edgecolors='darkgreen')
for label, x, y in zip(labels, T[-len(palabras):, 0], T[-len(palabras):, 1]):
    plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')

# Word embeddings con Gensim
Cargamos un conjunto de WE ya pre-entrenado con la API de Gensim:\
https://radimrehurek.com/gensim/downloader.html

In [ ]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

In [ ]:
api.info('glove-twitter-50')

In [ ]:
for model_name, model_data in sorted(api.info()['models'].items()):
    print(
        '{} ({} records):\n{}\n'.format(
            model_name,
            model_data.get('num_records', -1),
            model_data['description'],
        )
    )

In [ ]:
#cargamos el modelo deseado con
model = api.load("glove-wiki-gigaword-50")

In [ ]:
model

Los vectores de cada palabra del vocabulario se acceden como elementos de un diccionario en `model`

In [ ]:
dir(model)

In [ ]:
model.vector_size

In [ ]:
len(model.vocab)

In [ ]:
model.similarity('apple','pear')

Podemos usar los modelos cargados para ver los vectores de una palabra, buscar palabras similares o calcular analogías.\
Los modelos cargados son objetos de la clase `models.keyedvectors` (https://radimrehurek.com/gensim/models/keyedvectors.html)

In [ ]:
manzana = model["apple"]
type(manzana)

In [ ]:
manzana.shape

In [ ]:
model.most_similar("apple")

In [ ]:
palabra_rara = 'zamburiña'
try:
    vector = model[palabra_rara]
except KeyError:
    print(f"La palabra '{palabra_rara}' no aparece en este modelo")

### Analogías de word vectors con Gensim
Si *palabra_a* es a *palabra_b*, entonces *palabra_c* es a *??*\
Se calcula como el vector más cercano a (a-c)+b

In [ ]:
# hombre es a rey como mujer es a XX
# rey - hombre + mujer 
#https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar_cosmul
model.most_similar_cosmul(positive=['king','woman'],negative=['man'])

### Carga de otros modelos pre-entrenados en Gensim
En lugar de usar su API cargamos los modelos en formato texto. Hay varios modelos en Español en https://github.com/dccuchile/spanish-word-embeddings

In [ ]:
#carga de vectores en formato TXT
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec = '~/Downloads/fasttext-sbwc.100k.vec' #https://github.com/mquezada/starsconf2018-word-embeddings
cantidad = 100000
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [ ]:
wordvectors

In [ ]:
len(wordvectors.vocab)

In [ ]:
wordvectors['manzana'].shape

In [ ]:
wordvectors.most_similar(['manzana'])

In [ ]:
wordvectors.most_similar(positive=['rey','mujer'],negative=['hombre'], topn=3)

In [ ]:
wordvectors.most_similar(positive=['yerno','mujer'],negative=['hombre'], topn=3)

In [ ]:
# correr -> corría como saltar -> XX
wordvectors.most_similar(positive=['corrían','saltar'],negative=['correr'], topn=3)

In [ ]:
# Francia -> París como España -> XX
wordvectors.most_similar(positive=['parís','alemania'],negative=['francia'], topn=3)

In [ ]:
wordvectors.most_similar(positive=['hombre','malo'], topn=3)

### Modelos FastText
Los modelos de FastText se pueden cargar en formato texto (sólo palabras pre-entrenadas) o como modelo binario (calcula nuevas palabras a partir de su n-grama de caracteres)

In [ ]:
palabra_rara = 'pequeñín'
try:
    vector = wordvectors[palabra_rara]
except KeyError:
    print(f"La palabra '{palabra_rara}' no aparece en este modelo")

In [ ]:
del(wordvectors)

In [ ]:
# vectores de FastText desde el formato binario (lento, requiere mucha memoria)
        # descargado de https://fasttext.cc/docs/en/crawl-vectors.html
# ¡ojo, ocupan 4,5 GB!
from gensim.models.fasttext import load_facebook_vectors

wordvectors_file = '/Users/jovifran/Downloads/cc.es.300.bin'
wordvectors = load_facebook_vectors(wordvectors_file) #carga vectores pre-entrenados sólo

In [ ]:
wordvectors

In [ ]:
dir(wordvectors)

In [ ]:
'neorevolucionario' in wordvectors.vocab

In [ ]:
'pequeñín' in wordvectors.vocab

In [ ]:
wordvectors['neorevolucionario']

In [ ]:
wordvectors.most_similar('neorevolucionario')

In [ ]:
wordvectors.similarity('neorevolucionario', 'revolucionario')

## Entrenamiento de vectores propios
En lugar de usar vectores preentrenados los podemos entrenar con el modelo `word2vec` de Gensim

In [ ]:
import spacy
nlp = spacy.load('es_core_news_md')

In [ ]:
def normalizar_doc_tokenize(doc):
    '''Función que normaliza un texto cogiendo sólo
    las palabras en minúsculas mayores de 3 caracteres'''
    # separamos en tokens
    tokens = nlp(doc)
    # filtramos stopwords
    filtered_tokens = [t.lower_ for t in tokens if
                       len(t.text)>3 and
                       not t.is_space and
                       not t.is_punct]

    return filtered_tokens

In [ ]:
with open('cuento.txt', 'r', encoding = 'utf-8') as f:
    texto = f.readlines()
TOKENIZED_CORPUS = list(map(normalizar_doc_tokenize, texto))
len(TOKENIZED_CORPUS)

Calculamos los vectores de las palabras de nuestro corpus

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(TOKENIZED_CORPUS, #lista de documentos como lista de tokens
                               size=10,          #tamaño del vector
                               window=5,         #nº de términos adyacentes que usamos para el cálculo
                               min_count=2,      #nº mínimo de apariciones del término para contarlo
                               iter=100
                              )

#una vez entrenado el modelo nos quedamos con los vectores calculados
#si no se van a actualizar los vectores con nuevos documentos
model = model.wv
len(model.vocab)

Podemos listar todas las palabras del modelo

In [ ]:
model.vocab

In [ ]:
palabras = model.index2word
sorted(palabras)

Podemos ver el vector correspondiente a cualquier palabra del vocabulario

In [ ]:
print(model['personas'])

Podemos calcular la similitud entre palabras y buscar afinidades entre palabras

In [ ]:
print(model.similarity('cola', 'tripas'))

In [ ]:
print(model.similarity('ciudad', 'plaza'))

In [ ]:
print(model.similarity('cola', 'ciudad'))

In [ ]:
print(model.doesnt_match("cielo problemas catedral ciudad".split())) #palabra que no encaja en el contexto del resto

Visualizamos los vectores de nuestro vocabulario en 2 dimensiones con el algoritmo t-SNE

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

np.random.seed=123
palabras_sm = np.random.choice(palabras, 25, replace=False)
vectores = model[palabras_sm]

tsne = TSNE(n_components=2, random_state=0, n_iter=1000, perplexity=2)
np.set_printoptions(suppress=True)
T = tsne.fit_transform(vectores)
labels = palabras_sm

plt.figure(figsize=(14, 8))
plt.scatter(T[:, 0], T[:, 1], c='orange', edgecolors='r')
for label, x, y in zip(labels, T[:, 0], T[:, 1]):
    plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')

Ahora cargamos los vectores del modelo GloVe del módulo `spaCy` para nuestro corpus de prueba y representamos

In [ ]:
word_glove_vectors = np.array([nlp(word).vector for word in palabras])

In [ ]:
from sklearn.decomposition import PCA

tsne = PCA(n_components=2)
np.set_printoptions(suppress=True)
T = tsne.fit_transform(word_glove_vectors)
labels = palabras

plt.figure(figsize=(12, 6))
plt.scatter(T[:, 0], T[:, 1], c='orange', edgecolors='r')
for label, x, y in zip(labels, T[:, 0], T[:, 1]):
    plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')

In [ ]:
nlp.vocab['cola'].similarity(nlp.vocab['tripas'])

In [ ]:
nlp.vocab['ciudad'].similarity(nlp.vocab['plaza'])

In [ ]:
nlp.vocab['ciudad'].similarity(nlp.vocab['cola'])

# Vectores de documento (modelos semánticos)
Los vectores de documento recogen el sentido semántico de todo el documento como un vector de dimensines únicas.
## Modelos basados en *word embeddings*
Calcula el promedio de los *word embeddings* del documento para obtener un vector con sentido semántico de todo el documento.

In [ ]:
#Librería spaCy
nlp_en = spacy.load("en_core_web_md")
#El atributo vector del Doc o Span calcula el promedio de sus vectores de palabra

doc1 = nlp_en("I like salty fries and hamburgers.")
doc2 = nlp_en("Fast food tastes very good.")

In [ ]:
doc1[0].vector.shape

In [ ]:
doc1.vector.shape

In [ ]:
doc1[2:4].vector.shape

In [ ]:
# Similarity of two documents
print(doc1, "<->", doc2, doc1.similarity(doc2))
# Similarity of tokens and spans
french_fries = doc1[2:4]
burgers = doc1[5]
print(french_fries, "<->", burgers, french_fries.similarity(burgers))

In [ ]:
fast_food = doc2[0:2]
print(french_fries, "<->", fast_food, french_fries.similarity(fast_food))

In [ ]:
#Librería spaCy
#El atributo vector del Doc o Span calcula el promedio de sus vectores de palabra

doc1 = nlp("Me gustan las patatas fritas y las hamburguesas.")
doc2 = nlp("La comida rápida sabe muy bien.")

# Similarity of two documents
print(doc1, "<->", doc2, doc1.similarity(doc2))
# Similarity of tokens and spans
patatas_fritas = doc1[3:5]
hamburguesas = doc1[7]
print(patatas_fritas, "<->", hamburguesas, patatas_fritas.similarity(hamburguesas))

In [ ]:
comida_rapida = doc2[1:3]
print(patatas_fritas, "<->", comida_rapida, patatas_fritas.similarity(comida_rapida))

In [ ]:
#Librería Gensim
#calculamos a mano el vector promedio
import numpy as np
from numpy.linalg import norm # para normalizar datos

def to_vector(texto):
    tokens = texto.split()
    vec = np.zeros(300)
    for word in tokens:
        # si la palabra está la acumulamos
        if word in wordvectors:
            vec += wordvectors[word]
    return vec / norm(vec)

In [ ]:
#carga de vectores en formato TXT
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec = '~/Downloads/fasttext-sbwc.100k.vec'
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec)

In [ ]:
texto = 'me gustan los gatos'
to_vector(texto).shape

In [ ]:
#Calculamos similitud entre vectores de documentos
def similarity(texto_1, texto_2):
    vec_1 = to_vector(texto_1)
    vec_2 = to_vector(texto_2)
    sim = vec_1 @ vec_2 #producto punto de numpy
    return sim

In [ ]:
texto_1 = 'los felinos son lindos'
texto_2 = 'quiero comer pizza'

print(similarity(texto, texto_1))
print(similarity(texto, texto_2))

## Modelo basado en semántica de documentos
Hay un modelo propio de vectores de documentos (modelo `doc2vec` de Gensim)

In [ ]:
#doc2vec
from gensim.models.doc2vec import TaggedDocument


#Creamos corpus de entrada
all_docs = [TaggedDocument(tokens, [str(index)])
                for index, tokens in enumerate(TOKENIZED_CORPUS)]

In [ ]:
all_docs[0]

In [ ]:
from gensim.models import Doc2Vec

model_d2v = Doc2Vec(all_docs, vector_size=10,
                                 window=5, min_count=2, workers=4,
                                 alpha=0.025, 
                                 min_alpha=0.025,
                                 dm=0, dbow_words=0, dm_concat=0)

In [ ]:
model_d2v.train(all_docs, total_examples=len(all_docs), epochs=200)

In [ ]:
len(model_d2v.docvecs.doctags)

In [ ]:
#Una vez entrenado podemos ver el vector de cada documento
model_d2v.docvecs[2]

In [ ]:
#El modelo word vectors que usa internamente Doc2vec es igual que el de Word2Vec:
palabras = model_d2v.wv.index2word
len(palabras)

In [ ]:
model_d2v.wv['catedral']

In [ ]:
#Para calcular el docvec de un doc nuevo:
new_doc = 'La muchacha vio humo en la pastelería de la calle frente a la catedral'
new_d2v = model_d2v.infer_vector(normalizar_doc_tokenize(new_doc), steps=100)
new_d2v

Podemos relacionar el nuevo documento con el documento más parecido del corpus mediante su similitud coseno:

In [ ]:
model_d2v.docvecs.most_similar(positive=[new_d2v])

In [ ]:
' '.join(TOKENIZED_CORPUS[2])

In [ ]:
' '.join(TOKENIZED_CORPUS[4])